<h1> MODELO INFERENCIAL </h1>
<p> Para la parte inferencial definimos las variables con las que obtuvimos mejores resultados. </p>
<p> Variables tenidas en cuenta </p>

* Valor de la serie original
* Tendencia secuencial (0 , tamaño del DF)
* Var. Binarias dummies de meses
* Var. Binaria de periodo de covid
* Valor de la serie original desfasada a un periodo.

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

<h1>Funciones suplementarias</h1>

In [9]:

def predict_december(column,X_train_full, y_train_full, reg_tend_estac_lag_full):

    #Declaramos los meses de mayo en adelante
    meses = {
        'mes_5': 5, 'mes_6': 6, 'mes_7': 7, 'mes_8': 8, 'mes_9': 9,
        'mes_10': 10, 'mes_11': 11, 'mes_12': 12
    }
    
    #tendencia_meses = ['tendencia', 'mes_2', 'mes_3', 'mes_4', 'mes_5', 'mes_6', 'mes_7','mes_8', 'mes_9', 'mes_10', 'mes_11', 'mes_12', 'covid']

    for mes, mes_num in meses.items():
        # Crear la nueva fila para el mes
        nueva_fila = {
            'const': 1,
            column + "_lag1": y_train_full.values[-1],
            'tendencia': X_train_full['tendencia'].max() + 1,
            'mes_2': 0, 'mes_3': 0, 'mes_4': 0, 'mes_5': 0, 'mes_6': 0, 'mes_7': 0,
            'mes_8': 0, 'mes_9': 0, 'mes_10': 0, 'mes_11': 0, 'mes_12': 0,'covid': 0
        }
        nueva_fila[mes] = 1  # Activar el mes correspondiente

        # Convertir la nueva fila en un DataFrame
        nueva_fila_df = pd.DataFrame([nueva_fila], index=[X_train_full.index[-1] + pd.DateOffset(months=1)])

        # Predecir el valor para el mes
        prediccion = reg_tend_estac_lag_full.predict(nueva_fila_df)

        # Incorporar los datos de prueba al conjunto de entrenamiento
        X_train_full = pd.concat([X_train_full, nueva_fila_df])
        y_train_full = pd.concat([y_train_full, prediccion])

        reg_tend_estac_lag_full = sm.OLS(y_train_full, X_train_full).fit()  # Reajustar el modelo


    return X_train_full, y_train_full

def construir_graficos(column, predicciones, X_test, y_test, X_train_full, y_train_full, fecha_max, reg):

    #==== Graficar los resultados del modelo inicial
    fig1 = go.Figure()
    
    fig1.add_trace(go.Scatter(x=X_test.index, y=y_test, 
                              mode='lines+markers', 
                              name="Datos reales (10% de test)", 
                              line=dict(color='orange', width=2),
                              marker=dict(symbol='star', size=8)))

    fig1.add_trace(go.Scatter(x=X_test.index, y=predicciones, 
                              mode='lines+markers', 
                              name='Predicciones usando los datos de prueba', 
                              line=dict(color='green', width=2, dash='dash'),
                              marker=dict(symbol='circle', size=8)))

    fig1.update_layout(title=f'Predicción de facturación, categoría: {column} - Métrica: R² {reg.rsquared:.2f}',
                       xaxis_title='Fecha',
                       yaxis_title='Total Facturación',
                       legend_title='Leyenda',
                       hovermode='x unified')

    fig1.show()

    #=== Gráfico de las variaciones mes a mes por año
    list_years = np.unique(y_train_full.index.year)

    # Variaciones hasta el máximo histórico
    variaciones = y_train_full.pct_change()

    # Variaciones sin predicción
    variaciones_sin_prediccion = variaciones[variaciones.index <= fecha_max]

    # Variaciones de predicción || PONEMOS MAYOR IGUAL, PARA AGARRAR EL VALOR DEL MES ANTERIOR, ASI LOS GRAFICOS NO QUEDAN TAN "DISPAREJOS"
    variaciones_prediccion = variaciones[variaciones.index >= fecha_max]

    fig2 = go.Figure()

    for year in list_years:
        datos = variaciones_sin_prediccion[variaciones_sin_prediccion.index.year == year].dropna()
        if len(datos) == 0:
            continue

        primer_mes = datos.index.month[0]
        fig2.add_trace(go.Scatter(x=np.arange(primer_mes, primer_mes + len(datos)), 
                                  y=datos.values, 
                                  mode='lines+markers', 
                                  name=str(year)))

    # Para hacer el gráfico de las predicciones debemos hacer unas diferencias para el eje X
    if not variaciones_prediccion.empty:
        primer_mes_predicciones = variaciones_prediccion.index.month[0]
        fig2.add_trace(go.Scatter(x=np.arange(primer_mes_predicciones, primer_mes_predicciones + len(variaciones_prediccion)), 
                                  y=variaciones_prediccion.values, 
                                  mode='lines+markers', 
                                  name="Predicciones"))

    fig2.update_layout(title=f"Variaciones mensuales por año, categoría: {column}",
                       xaxis_title="Meses",
                       yaxis_title="Variaciones mensuales",
                       legend_title='Leyenda',
                       hovermode='x unified')

    fig2.show()

<h2>  Obtencion de datos </h2>

In [10]:
df = pd.read_excel('deflactacion_corrientes.xlsx')
df = df.drop('Unnamed: 0',axis=1)
df.set_index('fecha',inplace=True)

<h2> Tendencia </h2>
Vamos a añadir una caracteristica de tendencia, que consiste en una serie de valores secuenciales, de la forma lista(0, tamaño del dataframe).

In [11]:
df['tendencia'] = np.arange(1,len(df)+1)

<h2> Estacionalidad </h2>
Añadimos la variable de meses, esto con el objetivo de capturar la estacionalidad

In [12]:
df['mes'] = df.index.month #--> 1. Tomamos los meses

# Genera la dummie, tomando el DF como conjunto de datos, la columna 'meses' para generar la dummie
# y eliminamos el primer elemento para evitar la multicolinealidad. Por ultimo declaramos que sea numerica
# la variable binaria (trabajamos con 0 y 1).
df = pd.get_dummies(df,columns=['mes'], drop_first=True, dtype=int)

<h2> Caracteristica COVID </h2>
<p>Para añadir esta caracteristica, trabajamos con una dummie, que indica si el periodo de estudio forma parte de la pandemia.</p>
<p> Consideramos:</p>
<p> Fecha de inicio: 11 de Marzo 2020 - Declara la OMS</p>
<p> Fecha de "fin": Diciembre 2022 - Suposicion nuestra</p>


In [13]:
df['covid'] = ((df.index >= datetime(2020,3,11)) & (df.index <= datetime(2022,12,30))).astype(int)
df.columns

Index(['id_region_indec', 'id_provincia_indec', 'total_facturacion', 'bebidas',
       'almacen', 'panaderia', 'lacteos', 'carnes', 'verduleria_fruteria',
       'alimentos_preparados_rostiseria', 'articulos_limpieza_perfumeria',
       'indumentaria_calzado_textiles_hogar', 'electronica_hogar', 'otros',
       'tendencia', 'mes_2', 'mes_3', 'mes_4', 'mes_5', 'mes_6', 'mes_7',
       'mes_8', 'mes_9', 'mes_10', 'mes_11', 'mes_12', 'covid'],
      dtype='object')

<h2> Prediccion para cada categoria </h2>
<p>Buscaremos predecir con estas variables los valores futuros para cada categoria del supermercado.</p>
Para ello haremos un proceso iterativo que haga todo el proceso para cada categoria.

In [14]:
#Declaramos las columnas importantes que analizaremos
columnas = ['total_facturacion','bebidas','almacen','panaderia','lacteos','carnes','verduleria_fruteria',
            'alimentos_preparados_rostiseria','articulos_limpieza_perfumeria','indumentaria_calzado_textiles_hogar',
            'electronica_hogar','otros']

#Columnas de meses y covid que usaremos para cada calculo
tendencia_meses = ['tendencia', 'mes_2', 'mes_3', 'mes_4', 'mes_5', 'mes_6', 'mes_7','mes_8', 'mes_9', 'mes_10', 'mes_11', 'mes_12', 'covid']

#Fecha maxima del dataset - Se utiliza para posteriormente dividir los valores predichos a futuro de los datos que tenemos hasta ahora.
fecha_max = max(df.index)

"""
En este proceso abarcamos 3 pasos
1 - Ajustar el modelo usando un porcentaje de datos. Esto con fines descriptivos.
2 - Reajustar el modelo constantmente hasta llegar a DICIEMBRE DEL 2024.
3 - Graficar por cada categoria: SUMMARRY del modelo, PLOT de los resultados absolutos, comparacion de variaciones con otros años.
"""

for column in columnas:
    
    #DF auxiliar
    df_aux = pd.DataFrame()

    #Añadimos la variable objetivo - Posteriormente la eliminamos, pero la usamos para declarar nuestras X e Y.
    df_aux[column] = df[column]
    
    #Variable objetivo resagada a un perido
    df_aux[column+"_lag1"] = df[column].shift(1)

    #Asignamos al DF auxiliar las columnas del DF original, esto porque la tendencia y las dummies son las mismas para cada categoria.
    df_aux[tendencia_meses] = df[tendencia_meses]

    """
    DEFINICION DE X e Y:

    X: Nuestras variables independientes consistiran en:

        * La variable objetivo resagada a un periodo
        * La dummie de covid
        * Las dummies de meses

    Y: Nuestra variable dependiente consistiran en el valor de la serie original.

    Tambien declaramos que en una primera instancia haremos una division de datos.
    Usaremos el 90% de los datos para entrenar el modelo, y el restante 10% para testearlo.
    """

    #Lista completa de columnas de X
    column_lag1 = column + "_lag1"
    lista_completa = [column_lag1] + tendencia_meses

    #Declaramos X e Y
    X = df_aux.dropna(subset=[column+"_lag1"])[lista_completa]
    X = sm.add_constant(X)#--> Constante que añadimos (columna de 1)
    y = df_aux.dropna(subset=[column+"_lag1"])[column]

    #Division de datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    #Ajustamos el modelo
    reg = sm.OLS(y_train,X_train).fit()

    #Realizamos las predicciones con el conjunto de test --> Se usara posteriormente para hacer graficos
    predicciones = reg.predict(X_test)

    #Reajustamos el modelo con la totalidad de los datos
    X_train_full = pd.concat([X_train, X_test])
    y_train_full = pd.concat([y_train, y_test])


    #Este modelo se usara para predecir valores futuros
    reg_predict_future = sm.OLS(y_train_full,X_train_full).fit()

    #Realizamos predicciones hasta DICIEMBRE del 2024
    X_train_full, y_train_full = predict_december(column,X_train_full,y_train_full,reg_predict_future)

    print("****************************************************************************")
    print(f"=======  # SUMMARY DE LA CATEGORIA {column} # ======= ")
    print("\n")
    #print(reg_predict_future.summary())
    construir_graficos(column,predicciones,X_test,y_test,X_train_full,y_train_full,fecha_max,reg)

    print("****************************************************************************\n")

    


****************************************************************************
=======  # SUMMARY DE LA CATEGORIA total_facturacion # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA bebidas # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA almacen # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA panaderia # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA lacteos # ======= 




...

****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA articulos_limpieza_perfumeria # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA indumentaria_calzado_textiles_hogar # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA electronica_hogar # ======= 




****************************************************************************

****************************************************************************
=======  # SUMMARY DE LA CATEGORIA otros # ======= 




****************************************************************************

